See the following repositories for details  
https://github.com/tubo213/kaggle-child-mind-institute-detect-sleep-states

# Install additional libraries

In [1]:
!pip install hydra-core segmentation_models_pytorch==0.3.3 --no-index --find-links=/kaggle/input/ex-library

Looking in links: /kaggle/input/ex-library
Processing /kaggle/input/ex-library/hydra_core-1.3.2-py3-none-any.whl
Processing /kaggle/input/ex-library/segmentation_models_pytorch-0.3.3-py3-none-any.whl
Processing /kaggle/input/ex-library/pretrainedmodels-0.7.4.tar.gz (from segmentation_models_pytorch==0.3.3)
  Preparing metadata (setup.py) ... - done
Processing /kaggle/input/ex-library/efficientnet_pytorch-0.7.1.tar.gz (from segmentation_models_pytorch==0.3.3)
  Preparing metadata (setup.py) ... done
Processing /kaggle/input/ex-library/timm-0.9.2-py3-none-any.whl (from segmentation_models_pytorch==0.3.3)
Processing /kaggle/input/ex-library/munch-4.0.0-py2.py3-none-any.whl (from pretrainedmodels==0.7.4->segmentation_models_pytorch==0.3.3)
Processing /kaggle/input/ex-library/omegaconf-2.3.0-py3-none-any.whl (from hydra-core)
Processing /kaggle/input/ex-library/antlr4-python3-runtime-4.9.3.tar.gz (from hydra-core)
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-p

In [2]:
%cd /kaggle/input/cmi-code/

/kaggle/input/cmi-code


# Config

In [3]:
# Config
DURATION = 5760
DOWNSAMPLE_RATE = 2
PHASE = 'test'
EXP_NAME = 'exp001'

# Preprocess

In [4]:
!python -m run.prepare_data dir=kaggle phase=$PHASE

shape: (5, 5)
┌──────────────┬──────────┬───────────┬─────────────────────────┬────────────┐
│ series_id    ┆ anglez   ┆ enmo      ┆ timestamp               ┆ anglez_rad │
│ ---          ┆ ---      ┆ ---       ┆ ---                     ┆ ---        │
│ str          ┆ f32      ┆ f32       ┆ datetime[μs, UTC]       ┆ f32        │
╞══════════════╪══════════╪═══════════╪═════════════════════════╪════════════╡
│ 038441c925bb ┆ 0.322257 ┆ -0.192627 ┆ 2018-08-14 19:30:00 UTC ┆ 0.046019   │
│ 038441c925bb ┆ 0.32226  ┆ -0.194591 ┆ 2018-08-14 19:30:05 UTC ┆ 0.046021   │
│ 038441c925bb ┆ 0.322266 ┆ -0.193609 ┆ 2018-08-14 19:30:10 UTC ┆ 0.046024   │
│ 038441c925bb ┆ 0.32226  ┆ -0.196555 ┆ 2018-08-14 19:30:15 UTC ┆ 0.046021   │
│ 038441c925bb ┆ 0.32226  ┆ -0.194591 ┆ 2018-08-14 19:30:20 UTC ┆ 0.046021   │
└──────────────┴──────────┴───────────┴─────────────────────────┴────────────┘
['series_id', 'anglez', 'enmo', 'timestamp', 'anglez_rad']
[0.1GB(+0.0GB):0.2sec] Load series 
100%|█████████████████

# Inference

In [5]:
!python -m run.inference --cfg hydra dir=kaggle phase=$PHASE

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
hydra:
  run:
    dir: ${dir.output_dir}/${hydra.job.name}/${exp_name}/single
  sweep:
    dir: ${dir.output_dir}/${hydra.job.name}/${exp_name}
    subdir: run${hydra.job.num}
  launcher:
    _target_: hydra._internal.core_plugins.basic_launcher.BasicLauncher
  sweeper:
    _target_: hydra._internal.core_plugins.basic_sweeper.BasicSweeper
    max_batch_size: null
    params: null
  hydra_logging:
    version: 1
    formatters:
      simple:
        format: '[%(asctime)s][HYDRA] %(message)s'
    handlers:
      console:
        class: logging.StreamHandler
        formatter: simple
        stream: ext://sys.stdout
    root:
      level: INFO
      handlers:
      - console
    loggers:
      logging_example:
        level: DEBUG
    disable_

In [6]:
!python -m run.inference\
    dir=kaggle\
    num_workers=2\
    exp_name=$EXP_NAME\
    weight.run_name=single\
    batch_size=64\
    duration=$DURATION\
    downsample_rate=$DOWNSAMPLE_RATE\
    phase=$PHASE

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[2023-11-10 18:18:39,631][lightning_fabric.utilities.seed][INFO] - Global seed set to 42
[1.5GB(+0.0GB):0.0sec] load test dataloader 
load weight from "/kaggle/input/cmi-model/exp001/single/best_model.pth"
[3.6GB(+2.1GB):8.1sec] load model 
inference: 100%|██████████████████████████████████| 1/1 [00:06<00:00,  6.35s/it]
[4.7GB(+1.1GB):6.4sec] inference 
[4.8GB(+0.0GB):0.0sec] make submission 


In [7]:
# いらないファイル、フォルダ削除
!rm -rf /kaggle/working/processed_data
!rm -rf /kaggle/working/output